# Final

In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [18]:
data = pd.read_csv("C:\\Users\\Almas\\Downloads\Malware.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 35 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   hash               100000 non-null  object
 1   millisecond        100000 non-null  object
 2   classification     100000 non-null  object
 3   state              100000 non-null  object
 4   usage_counter      100000 non-null  object
 5   prio               100000 non-null  object
 6   static_prio        100000 non-null  object
 7   normal_prio        100000 non-null  object
 8   policy             100000 non-null  object
 9   vm_pgoff           100000 non-null  object
 10  vm_truncate_count  100000 non-null  object
 11  task_size          100000 non-null  object
 12  cached_hole_size   100000 non-null  object
 13  free_area_cache    100000 non-null  object
 14  mm_users           100000 non-null  object
 15  map_count          100000 non-null  object
 16  hiwater_rss        10

In [37]:
label_encoder = LabelEncoder()
data['classification'] = label_encoder.fit_transform(data['classification'])

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
X = data.drop(columns=['classification', 'hash'])  
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [41]:
y = data['classification']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [44]:
model = Sequential([
    SimpleRNN(64, activation='relu', input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

D:\programs\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [45]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [46]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train_rnn, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8654 - loss: 0.3018 - val_accuracy: 0.9934 - val_loss: 0.0305
Epoch 2/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9832 - loss: 0.0499 - val_accuracy: 0.9964 - val_loss: 0.0124
Epoch 3/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9910 - loss: 0.0279 - val_accuracy: 0.9975 - val_loss: 0.0068
Epoch 4/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9938 - loss: 0.0207 - val_accuracy: 0.9983 - val_loss: 0.0064
Epoch 5/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9946 - loss: 0.0156 - val_accuracy: 0.9986 - val_loss: 0.0038
Epoch 6/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9953 - loss: 0.0144 - val_accuracy: 0.9989 - val_loss: 0.0062
Epoch 7/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9957 - loss: 0.0138 - val_accuracy: 0.9986 - val_loss: 0.0045
Epoch 8/20
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9952 - loss: 0.0138 - 

In [47]:
y_pred = (model.predict(X_test_rnn) > 0.5).astype("int32").ravel()
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 730us/step
Accuracy: 0.99905
F1 Score: 0.9990479530991633
Recall: 0.9998996990972919
Precision: 0.9981976569540403
